In [ ]:
!pip install jupyter-dash

In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

<ipython-input-1-6fe743471c61>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-6fe743471c61>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-11-06 16:24:00--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: 'spacex_launch_dash.csv.3'

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-11-06 16:24:01 (161 MB/s) - 'spacex_launch_dash.csv.3' saved [3033/3033]



In [3]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

--2021-11-06 16:24:02--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2110 (2.1K) [text/x-python]
Saving to: 'spacex_dash_app.py.3'

spacex_dash_app.py. 100%[===================>]   2.06K  --.-KB/s    in 0s      

2021-11-06 16:24:03 (71.9 MB/s) - 'spacex_dash_app.py.3' saved [2110/2110]



In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [5]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [6]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [9]:
# Create a dash application
app = JupyterDash(__name__)
server = app.server

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                    options=[{'label': 'All Sites', 'value': 'ALL'},
                                             {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                             {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                             {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                             {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                    value='ALL',
                                    placeholder="Please select a Launch Site",
                                    searchable=True
                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    #filtered_df = spacex_df
    if entered_site == 'ALL':
        title_pie= f'Success Launches for site: {entered_site}'
        spacex_df_class=spacex_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(spacex_df_class, values='class count',names='class', title=title_pie)
    
    else:
        dropdown_pie= spacex_df[spacex_df['Launch Site'] == entered_site]
        dropdown_pie_class = dropdown_pie.groupby(['Launch Site','class']).size().reset_index(name='class count')
        title_pie = f"Success Launches for site: {entered_site}"       
        fig = px.pie(dropdown_pie_class, values='class count', names='class', title=title_pie)

    return fig
        # return the outcomes piechart for a selected site


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])

def build_scatter(site_dropdown,slider_range):
    
    low, high = slider_range
    slide = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    dropdown_scatter=spacex_df[slide]

    if site_dropdown == 'ALL':
        
        #Display all values for the variable
        #dropdown_scatter= spacex_df[spacex_df['Launch Site'] == site_dropdown]
        #dropdown_scatter_all = pd.DataFrame(dropdown_scatter(['Launch Site']))
        title_scatter = f"Payload and Booster Versions for site: {site_dropdown}"
        fig = px.scatter(dropdown_scatter,x="Payload Mass (kg)", y="class", color="Booster Version Category", title=title_scatter)

    else:
        dropdown_scatter1= dropdown_scatter[dropdown_scatter['Launch Site'] == site_dropdown]
        title_scatter = f"Payload and Booster Versions for site {site_dropdown}"
        fig = px.scatter(dropdown_scatter1, x="Payload Mass (kg)", y="class", color="Booster Version Category", title=title_scatter)
        
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


Dash app running on http://127.0.0.1:8050/


In [ ]:
spacex_df_class=spacex_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
spacex_df_class.head()